# Get PS1 DR2 Quasar data 

Taking the 9258  S82 QSO from SDSS (DBQSO). Seek to find the PS1, PTF, CRTS, data.   Copied only what was necessary to get PS1 light curves from `Crossmatch_SDSS_to_all.ipynb` ... 

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import os
import pandas as pd 
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import  hstack
from astropy.table import  vstack
from astropy.table import Column
from astropy.table import join
from matplotlib import rcParams 
rcParams['ytick.labelsize'] = 15
rcParams['xtick.labelsize'] = 15
rcParams['axes.labelsize'] = 20
rcParams['axes.linewidth'] = 2
rcParams['font.size'] = 15
rcParams['axes.titlesize'] = 18

Get the DR7 list of SDSS S82 quasars :
    
    

In [ ]:
# Upload to CasJobs a table with SDSS_DR7 qso  ra,dec 
File = '../../CRTS_PROJECT/catalogs_SDSS/DB_QSO_S82.dat'
colnames = ['dbID', 'ra', 'dec', 'SDR5ID', 'M_i', 'M_i_corr', 'redshift',
                   'mass_BH', 'Lbol', 'u', 'g', 'r', 'i', 'z', 'Au']

# 9258 QSO ....
SDSS_DB_QSO= Table.read(File, format='ascii', names = colnames)

dbqso_radec = SDSS_DB_QSO[['dbID', 'ra', 'dec']].copy()
dbqso_radec.write('sdss_s82_dr7_qso.txt', format='ascii')

PS1 DR2 links 

- an overview of everything: https://outerspace.stsci.edu/display/PANSTARRS/How+to+retrieve+and+use+PS1+data 
- there is forced photometry  https://outerspace.stsci.edu/display/PANSTARRS/PS1+Forced+photometry+of+sources  
- as well as single epoch detections  https://outerspace.stsci.edu/display/PANSTARRS/PS1+Detection+properties
- can also use a high-level GUI  Catalog search https://catalogs.mast.stsci.edu/panstarrs/ 
- many useful sample queries here https://outerspace.stsci.edu/display/PANSTARRS/PS1+Sample+queries#PS1Samplequeries-Obtainlightcurvesforagivensetofobjects  - that's perfect, because it walks me step by step (literally!!!  )  

-this shows transmisson curves and different filter characteristics ... http://svo2.cab.inta-csic.es/svo/theory/fps3/index.php?id=PAN-STARRS/PS1.g&&mode=browse&gname=PAN-STARRS&gname2=PS1  
- and the data used to make the transmission curves for the PS1 filters http://ipp.ifa.hawaii.edu/ps1.filters/apj425122t3_mrt.txt 



**Querying CAS Jobs  : **

* manually remove the comment section
* log on to  http://mastweb.stsci.edu/ps1casjobs/SubmitJob.aspx  
* upload that table to CasJobs, calling it `S82_DR7_QSO`
* in query, set context to `PanStarrs1`

* execute the following query (based on https://outerspace.stsci.edu/display/PANSTARRS/PS1+Sample+queries#PS1Samplequeries-Obtainlightcurvesforagivensetofobjects )  to get objIDs  within 2 arcsec from each object  ( setting context to PS1 DR2... ) 

 
   
    SELECT d.dbID, d.ra, d.dec,
    o.objID, 
    o.raMean, o.decMean, o.raMeanErr, o.decMeanErr,
    o.qualityFlag,
    o.nDetections, o.ng, o.nr, o.ni, o.nz,o.ny,
    o.gFlags, o.gQfPerfect,
    o.rFlags, o.rQfPerfect,
    o.iFlags, o.iQfPerfect,
    o.zFlags, o.zQfPerfect,
    o.yFlags, o.yQfPerfect,
    soa.primaryDetection, soa.bestDetection
     INTO mydb.[S82_DR7_QSO_PS1_objID]
     FROM mydb.[S82_DR7_QSO] d
    CROSS APPLY dbo.fGetNearbyObjEq(d.ra, d.dec, 2.0/60.0) as x
    JOIN MeanObjectView o on o.ObjID=x.ObjId
    LEFT JOIN StackObjectAttributes AS soa ON soa.objID = x.objID
    WHERE o.nDetections>5
    AND soa.primaryDetection>0
    AND o.gQfPerfect>0.85 and o.rQfPerfect>0.85 and o.iQfPerfect>0.85 and o.zQfPerfect>0.85
    AND (o.rmeanpsfmag - o.rmeankronmag < 0.05)


  
* and then run the query to get all the detection data for these objIDs .... 


    SELECT o.objID, o.raMean, o.decMean,
    d.ra, d.dec, d.raErr, d.decErr,
    d.detectID, d.obstime, d.exptime, d.airmass, d.psfflux, d.psffluxErr, d.psfQf, d.psfQfPerfect, d.psfLikelihood, d.psfChiSq, d.extNSigma, d.zp, d.apFlux, d.apFluxErr,
    d.imageID, d.filterID,
    d.sky, d.skyerr, d.infoflag, d.infoflag2, d.infoflag3,
    o.qualityFlag,
    o.nDetections, o.primaryDetection, o.bestDetection
     INTO mydb.[S82_DR7_QSO_PS1_det]
     FROM mydb.[S82_DR7_QSO_PS1_objID] o
    JOIN Detection d on d.ObjID = o.ObjID
    
    
* finally run this query to get also all forced photometry for these detections : 

    
    SELECT
    o.objID, o.raMean, o.decMean,
    fwm.detectID, fwm.obstime, fwm.exptime, fwm.airmass, fwm.Fpsfflux, fwm.FpsffluxErr, fwm.FpsfQf, fwm.FpsfQfPerfect, fwm.FpsfChiSq, fwm.zp, fwm.FapFlux, fwm.FapFluxErr,
    fwm.forcedWarpID, fwm.filterID,
    fwm.Fsky, fwm.Fskyerr, fwm.Finfoflag, fwm.Finfoflag2, fwm.Finfoflag3
     INTO mydb.[S82_DR7_QSO_PS1_fp]
     FROM mydb.[S82_DR7_QSO_PS1_objID] o
    JOIN ForcedWarpMeasurement fwm on fwm.ObjID = o.ObjID
    
    
    
* all results can be easily previewed, and downloaded as csv  